In [54]:
import cv2  # Import the OpenCV library
import numpy as np  # Import Numpy library
from scipy.spatial.transform import Rotation as R
import math  # Math library

In [55]:
markerSize = 0.0526*1000 # millimetres

# Calibration parameters yaml file
calFile = 'calibration_chessboard.yaml'
cv_file=cv2.FileStorage(calFile, cv2.FILE_STORAGE_READ)
mtx=cv_file.getNode('K').mat()
dist=cv_file.getNode('D').mat()
cv_file.release()

In [56]:

def euler_from_quaternion(x, y, z, w):
    """
    Convert a quaternion into euler angles (roll, pitch, yaw)
    roll is rotation around x in radians (counterclockwise)
    pitch is rotation around y in radians (counterclockwise)
    yaw is rotation around z in radians (counterclockwise)
    """
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll_x = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    pitch_y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw_z = math.atan2(t3, t4)

    return roll_x, pitch_y, yaw_z  # in radians


In [57]:
def my_estimatePoseSingleMarkers(corners, marker_size, mtx, distortion):
    '''
    This will estimate the rvec and tvec for each of the marker corners detected by:
       corners, ids, rejectedImgPoints = detector.detectMarkers(image)
    corners - is an array of detected corners for each detected marker in the image
    marker_size - is the size of the detected markers
    mtx - is the camera matrix
    distortion - is the camera distortion matrix
    RETURN list of rvecs, tvecs, and trash (so that it corresponds to the old estimatePoseSingleMarkers())
    '''
    marker_points = np.array([[-marker_size / 2, marker_size / 2, 0],
                              [marker_size / 2, marker_size / 2, 0],
                              [marker_size / 2, -marker_size / 2, 0],
                              [-marker_size / 2, -marker_size / 2, 0]], dtype=np.float32)
    trash = []
    rvecs = []
    tvecs = []
    for c in corners:
        nada, R, t = cv2.solvePnP(marker_points, c, mtx, distortion, False, cv2.SOLVEPNP_IPPE_SQUARE)
        rvecs.append(R)
        tvecs.append(t)
        trash.append(nada)
    return rvecs, tvecs, trash

In [58]:
boardVertical=.084*1000
boardHorizontal=.104*1000
markerPositions={25:(markerSize/2, markerSize/2, 0), 26:(markerSize/2, markerSize/2+boardVertical, 0), 20:(markerSize/2+boardHorizontal, markerSize/2, 0), 27:(markerSize/2+boardHorizontal, markerSize/2+boardVertical, 0)}

#print(markerPositions[25])

In [59]:


aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_7X7_50)
parameters = cv2.aruco.DetectorParameters()
# Create the ArUco detector
detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

cap=cv2.VideoCapture(0)
while True:
  ret, frame=cap.read()
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  corners, marker_ids, rejected = detector.detectMarkers(gray)
  if marker_ids is not None:
    cv2.aruco.drawDetectedMarkers(frame, corners, marker_ids)
    rvecs, tvecs, obj_points = my_estimatePoseSingleMarkers(corners,
                                                            markerSize,
                                                            mtx,
                                                            dist)
    for i, marker_id in enumerate(marker_ids):
        rvec=rvecs[i]
        tvec=tvecs[i]
    
        rvec_flip=-1*rvec
        tvec_flip=-1*tvec
        Rmat, J = cv2.Rodrigues(rvec_flip)
        t_RW = np.dot(Rmat, tvec_flip)
        t_RW = t_RW.flatten()
        r=R.from_matrix(Rmat[0:3, 0:3])
        quat=r.as_quat()
    
        rot_x=quat[0]
        rot_y=quat[1]
        rot_z=quat[2]
        rot_w=quat[3]
        roll_x, pitch_y, yaw_z = euler_from_quaternion(rot_x, rot_y, rot_z, rot_w)
        yaw=yaw_z*180/np.pi
    
        marker_posn=markerPositions[marker_id[0]]
        marker_x=float(marker_posn[0])
        marker_y=float(marker_posn[1])
        marker_str=f'Marker {marker_id}, x: {marker_x:.3f}, y: {float(marker_y):.3f}'
        marker_textlocn= corners[i][0][1]
        m_loc=int(marker_textlocn[0]), int(marker_textlocn[1])
        cv2.putText(frame, marker_str, m_loc, cv2.FONT_HERSHEY_PLAIN,1, (0,255, 255), 2)

        tvec_str=f'x: {t_RW[0]:.3f} y: {t_RW[1]:.3f} yaw: {yaw:.3f}'
        pos=corners[i][0][0]
        #print(pos)
        cv2.putText(frame, tvec_str, (int(pos[0]), int(pos[1])), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
        
        poseX=marker_x+t_RW[0]
        poseY=marker_y+t_RW[1]
        pose_textloc= corners[i][0][2]
        p_loc=int(pose_textloc[0]), int(pose_textloc[1])
        p_str=f'x: {poseX:.3f} y: {poseY:.3f}'
        cv2.putText(frame, p_str, p_loc, cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)

        cv2.drawFrameAxes(frame, mtx, dist, rvec, tvec, 5)
  cv2.circle(frame, (int(frame.shape[1]/2), int(frame.shape[0]/2)), 2, (0, 0, 255), -1)
  cv2.imshow('frame', frame)
  if cv2.waitKey(1) & 0xFF==ord('q'):
    break

cap.release()
cv2.destroyAllWindows()